# Защита персональных данных

Цель: защитить данные клиентов страховой компании.
- Разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.
- Дать обоснование корректности его работы. При преобразовании, качество моделей машинного обучения не должно ухудшаться
- Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from numpy import linalg as LA

float_formatter = "{:.3f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
pd.options.display.float_format ='{:,.2f}'.format


## Предобработка и оценка данных

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.00,"49,600.00",1,0
1,0,46.00,"38,000.00",1,1
2,0,29.00,"21,000.00",0,0
3,0,21.00,"41,700.00",2,0
4,1,28.00,"26,100.00",0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


**Комментарий:** данные без пропусков. Заработную плату  и  возраст перевелем в int.

In [4]:
columns = data.columns
data = data.astype('int')

In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,"5,000.00","5,000.00","5,000.00","5,000.00","5,000.00"
mean,0.50,30.95,"39,916.36",1.19,0.15
std,0.50,8.44,"9,900.08",1.09,0.46
min,0.00,18.00,"5,300.00",0.00,0.00
25%,0.00,24.00,"33,300.00",0.00,0.00
50%,0.00,30.00,"40,200.00",1.00,0.00
75%,1.00,37.00,"46,600.00",2.00,0.00
max,1.00,65.00,"79,000.00",6.00,5.00


**Комментарий:** данные распределены нормально, каких либо отклонений и выбросов  нет.

## Умножение матриц

**Проведем теоретическое  исследование:** <br/>
При перемножении признаков умножают на обратимую матрицу. Изменяется ли качество линейной регрессии? Её можно обучить заново?
- Если изменится. Привести примеры матриц.
- Если не изменится. Указать, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

In [6]:
features = data.drop('Страховые выплаты',axis=1)
target = data['Страховые выплаты']

- Проверим работу формулу определения $w$ и из sklearn

In [7]:
w0 = np.ones((features.shape[0],1))
X = np.concatenate((w0,features),axis=1)
y = np.array(target)
w  = np.linalg.inv(X.T@X)@X.T@y
display(w[1:])

model = LinearRegression()
model.fit(features,target)
model.coef_

array([0.008, 0.036, -0.000, -0.014])

array([0.008, 0.036, -0.000, -0.014])

**Комментарий:** Расчет коэффициентов одинаковый. Мы на правильном пути.

**Обозначения:**

- $X$ — матрица признаков (+ нулевой столбец,который состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- Предсказания:<br/>
$a = Xw$<br/>
- Задача обучения:<br/>
$w = \arg\min_w MSE(Xw, y)$<br/>
- Формула обучения:<br/>
$w = (X^T X)^{-1} X^T y$<br/>

Необходимо доказать , что $a' = X'w' = Xw = a$.<br/>Заменим $X = XP$, и подставим в формулу обучения, получим $w' = ((XP)^T XP)^{-1} (XP)^T y$

Воспользуемяся свойствами матриц <br/>
1. $(AB)^{-1} = B^{-1}A^{-1}$<br/>
2. $(AB)^T = B^TA^T$ <br/>
3. $AA^{-1} = E$ (Единичная матрица) <br/>
Перейдем к преобразованию

$w'= ((XP)^T XP)^{-1} (XP)^T y$ <br/>
<br/>
Используя свойства транспонированных матриц представим транспонированное произведение, как произведение транспонированных матриц, взятых в обратном порядке $(AB)^T = B^TA^T$<br/>
<br/>
$((P^T)(X^TXP))^{-1}P^TX^Ty$ (2.1) 

* $P^T$ - квадратная матрица, утверждение вытекает из условия задачи.<br/>
* $(X^TXP)$ - также квадратная матрица, так как произведение матрицы и транспонированной матрицы - квадратная матрица. Умножение квадратных матриц  дает квадраную матрицу. Значит для квадратных обратимых матриц мы можем применить свойство обратных матриц $(AB)^{-1} = B^{-1}A^{-1}$ <br/>

Продолжим трансформации формулы (2.1) <br/>
 <br/>
$(X^TXP)^{-1} (P^T)^{-1} P^TX^Ty = ((X^TX)P)^{-1}EX^Ty = P^{-1}((X^TX)^{-1}X^Ty) =P^{-1}w
$<br/>


$a' = X'w' = XPP^{-1}w =Xw= a$

**Вывод:** При умножение признаков на обратимую матрицу, качество линейной  регрессии измениться не  должно.


## Алгоритм преобразования

Для защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу, которая будет генерироваться случайным образом. Чтобы матрица была обратимой, она как минимум должна быть квадратной<br/>
1. Составим матрицу
2. Проверка матрицы на обратимость. Проверка наличия детерминанта
3. формирование матрицы с преобразованными признаками 
4. Применение алгоритма на преобразованных данных

**Обоснование**

Матрица  должна быть размерностью n x n , где n - количество признаков регрессии. Как  ранее упоминалось, что Обратная матрица  существует только для квадратных невырожденных матриц (определитель которых не равен нулю).


## Проверка алгоритма

Так как будем работать с линейной регрессией, то требуется масштабирование даннных.

In [8]:
scaler = StandardScaler()
scaler.fit(features)
features_scaler = scaler.transform(features)

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features_scaler, target, test_size=0.25, random_state=393)

In [10]:
line_regres = LinearRegression()
line_regres.fit(features_train, target_train)
prediction = line_regres.predict(features_test)
r2 =  r2_score(target_test, prediction)
print('R2 - score:',r2)

R2 - score: 0.4157845318586466


**Вывод:** метрика R2 до преобразования составляет, 0.4157845314096005. После преобразований она должна остаться такой же.

Создадим функцию преобразования матрицы признаков.

In [11]:
def create_crypto_matrix(features):
    try:  
        np.random.seed(393)
        n = features.shape[1]
        random_matrix= np.random.randint(-10,10,(n,n))
        random_matrix = LA.inv(random_matrix)
        crypto_matrix = features@random_matrix
        return crypto_matrix
    except:
        print ('Матрица является вырожденной')

In [12]:
display(features.head())
display(pd.DataFrame(features_scaler).head())
display(pd.DataFrame(create_crypto_matrix(features)).head())

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


,0,1,2,3
0,1.00,1.19,0.98,-0.18
1,-1.00,1.78,-0.19,-0.18
2,-1.00,-0.23,-1.91,-1.09
3,-1.00,-1.18,0.18,0.74
4,1.00,-0.35,-1.40,-1.09


,0,1,2,3
0,"-2,049.28","-5,275.52","-1,935.25","-1,897.40"
1,"-1,571.38","-4,043.00","-1,482.77","-1,453.23"
2,-868.77,"-2,234.55",-819.46,-803.03
3,"-1,721.69","-4,434.07","-1,627.03","-1,595.48"
4,"-1,078.90","-2,776.61","-1,018.33",-998.30


**Вывод:** по порядку  сверху вниз идут текущий дата фрейм признаков, далее отмасштабированный и последний перемноженные на обратимую матрицу. Очевидно, что понять суть значений невозможно.

In [13]:
crypto_matrix = create_crypto_matrix(features)
scaler_crypto = StandardScaler()
scaler_crypto.fit(crypto_matrix)
features_scaler_crypto = scaler_crypto.transform(crypto_matrix)
features_train_crypto, features_test_crypto, target_train_crypto, target_test_crypto = train_test_split(features_scaler_crypto, target, test_size=0.25, random_state=393)


In [14]:
line_regres_crypto = LinearRegression()
line_regres_crypto.fit(features_train_crypto, target_train_crypto)
prediction = line_regres_crypto.predict(features_test_crypto)
r2 =  r2_score(target_test_crypto, prediction)
print('R2 - score:',r2)

R2 - score: 0.4157845318586437


**Вывод:** Метрика R2 до и после преобразования  составляет 0.415784531409, что говорит о том, используя  обратимую матрицу, как инструмент шифрования данных, качество линейной регресии при этом не меняется.

# Итог:
По результатам работы:
1. Сформировали алгоритм  шифрования  данных
2. Качество модели после использования алгоритма осталось неизменным.
3. Шифрование удалось, данные клиентов компании защищены